model, use_dfl 인자로 값을 전달 받는다. hyperparameter의 cls_pw는 클래스 손실 함수의 pos_weight의 값을 의미한다. label_smoothing 키 값이 설정되어 있으면 class label에 대한 positive, negative soft label이 생성성 된다.

```python
class ComputeLoss:
    def __init__(self, model, use_dfl=True):
        ...
        # Define criteria
        BCEcls = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([h["cls_pw"]], device=device), reduction='none')

        # Class label smoothing https://arxiv.org/pdf/1902.04103.pdf eqn 3
        self.cp, self.cn = smooth_BCE(eps=h.get("label_smoothing", 0.0))  # positive, negative BCE targets
        ...
```

fl_gamma 키 값이 0 보다 큰 값으로 설정되어 있으면 BCEcls를 FocalLoss로 생성한다.

```python
        ...
        # Focal loss
        g = h["fl_gamma"]  # focal loss gamma
        if g > 0:
            BCEcls = FocalLoss(BCEcls, g)
        ...
```

```python
        ...
        self.assigner = TaskAlignedAssigner(topk=int(os.getenv('YOLOM', 10)),
                                            num_classes=self.nc,
                                            alpha=float(os.getenv('YOLOA', 0.5)),
                                            beta=float(os.getenv('YOLOB', 6.0)))
        self.bbox_loss = BboxLoss(m.reg_max - 1, use_dfl=use_dfl).to(device)
        ...
```